In [3]:
import pandas as pd

In [13]:
def TimeStampMaker(input_df):
    input_df['Timestamp'] = input_df['Month'].astype(str) + "-" + input_df['Day'].astype(str) +" "\
                                                        + input_df['Hour'].astype(str)+ ":" + input_df['Minute'].astype(str)
    
    input_df.set_index('Timestamp', inplace = True)    

def PSMTrainingSetMaker(target_year, input_years_prior, city,half_hour_interval = False):
    
    num_years = input_years_prior
    
    if city == 'Santa_Monica':
        base_filepath = 'Santa_Monica/83160_34.01_-118.46_'
        
    elif city == 'San_Francisco':
        base_filepath = 'San_Francisco/120637_37.61_-122.42_'
        
    elif city == 'Boulder':
        base_filepath = 'Boulder/150658_40.13_-105.22_'
    
    elif city == 'Eugene':
        base_filepath = 'Eugene/198107_44.05_-123.06_'
    
    target_df = pd.read_csv('./data_psm_raw/' + str(base_filepath) + str(target_year) + '.csv',header = 2)
    
    TimeStampMaker(target_df)
    
    target_df['Target' + '_DNI'] = target_df['DNI'].copy()
    
    ## Target Year Status printout ##
    base_header = pd.read_csv('./data_psm_raw/' + str(base_filepath) + str(target_year) + '.csv',nrows = 3).values
    loc_header = str(base_header[0][1])+' '+str(base_header[2][0])+' '+str(base_header[0][5])+' '+str(base_header[0][6])
    print('Target = ' + str(city) + ' ' + loc_header + " Adding Years prior: " + str(num_years))
    
    if half_hour_interval:
        loop_df = target_df.loc[:,['Target' + '_DNI', 'Solar Zenith Angle']].copy()
    else:
        loop_df = target_df.loc[:,['Target' + '_DNI', 'Solar Zenith Angle', 'Minute']].copy()
    
    year_count = 0
    for year in range(target_year - num_years, target_year):
        
        new_df = pd.read_csv('./data_psm_raw/' + str(base_filepath) + str(year) + '.csv',header = 2)
        
        TimeStampMaker(new_df)
        
        features = ['Clearsky DNI','Clearsky DHI', 'Relative Humidity', 'Cloud Type']
        
        new_df = new_df.loc[:,features].copy()
        
#         new_df.drop(['Year', 'Month', 'Day', 'Hour', 'Minute'], axis =1,inplace = True)
    
        new_df['Cloud Type'].fillna(10, inplace = True)  # 10 is the value for "unknown"
        
        new_df['Cloud Type'] = new_df['Cloud Type'].map(lambda x: 1 if x <= 1 else 0)
        
#         new_df = pd.get_dummies(new_df, columns = ['Cloud Type'])
        new_df.rename({"Cloud Type": 'Clear Sky'})
        
        new_df.columns = ['t-'+str(year_count+1) + "_" + col for col in new_df.columns]

        loop_df = pd.merge(left = loop_df, right = new_df, how = 'right', left_index = True, right_index = True)
        
        ## Status print-out ##
        base_header = pd.read_csv('./data_psm_raw/' + str(base_filepath) + str(year) + '.csv',nrows = 3).values
        loc_header = str(base_header[0][1])+' '+str(base_header[2][0])+' '+str(base_header[0][5])+' '+str(base_header[0][6])
        print(str(city)+' '+loc_header)
        
        year_count += 1
        
    print(city, target_year, "Number of Years Prior:", year_count)
    
    if half_hour_interval:
        return loop_df
    
    else:
        return loop_df.loc[loop_df['Minute'] == 0,:].drop('Minute', axis = 1)

In [16]:
target_years_prior = 14

for location in ['Santa_Monica']:
    for time_target in [2016, 2015]:
        
        filepath = './data_psm_pkls/' + str(location) + "_" + str(time_target) + "_" \
                                                            + str(target_years_prior) + "yearsprior_30min.pkl"
        
        df_1 = PSMTrainingSetMaker(time_target, 
                            target_years_prior, city = location, 
                            half_hour_interval = True)
        
        pd.to_pickle(df_1, filepath)
        
        print("\nSaved to " + filepath + "\n")
        
        
        filepath = './data_psm_pkls/' + str(location) + "_" + str(time_target) + "_" \
                                                            + str(target_years_prior) + "yearsprior_1hour.pkl"
        
        df_2 = PSMTrainingSetMaker(time_target, target_years_prior, 
                            city = location, 
                            half_hour_interval = False)
        
        pd.to_pickle(df_2, filepath)
        
        print("\nSaved to " + filepath + "\n")

Target = Santa_Monica 83160 2016 34.01 -118.46 Adding Years prior: 14
Santa_Monica 83160 2002 34.01 -118.46
Santa_Monica 83160 2003 34.01 -118.46
Santa_Monica 83160 2004 34.01 -118.46
Santa_Monica 83160 2005 34.01 -118.46
Santa_Monica 83160 2006 34.01 -118.46
Santa_Monica 83160 2007 34.01 -118.46
Santa_Monica 83160 2008 34.01 -118.46
Santa_Monica 83160 2009 34.01 -118.46
Santa_Monica 83160 2010 34.01 -118.46
Santa_Monica 83160 2011 34.01 -118.46
Santa_Monica 83160 2012 34.01 -118.46
Santa_Monica 83160 2013 34.01 -118.46
Santa_Monica 83160 2014 34.01 -118.46
Santa_Monica 83160 2015 34.01 -118.46
Santa_Monica 2016 Number of Years Prior: 14

Saved to ./data_psm_pkls/Santa_Monica_2016_14yearsprior_30min.pkl

Target = Santa_Monica 83160 2016 34.01 -118.46 Adding Years prior: 14
Santa_Monica 83160 2002 34.01 -118.46
Santa_Monica 83160 2003 34.01 -118.46
Santa_Monica 83160 2004 34.01 -118.46
Santa_Monica 83160 2005 34.01 -118.46
Santa_Monica 83160 2006 34.01 -118.46
Santa_Monica 83160 2007 34

In [15]:
pd.read_pickle('./data_psm_pkls/Santa_Monica_2016_14yearsprior_1hour.pkl').head()

,Target_DNI,Solar Zenith Angle,t-1_Clearsky DNI,t-1_Clearsky DHI,t-1_Relative Humidity,t-1_Cloud Type,t-2_Clearsky DNI,t-2_Clearsky DHI,t-2_Relative Humidity,t-2_Cloud Type,...,t-12_Relative Humidity,t-12_Cloud Type,t-13_Clearsky DNI,t-13_Clearsky DHI,t-13_Relative Humidity,t-13_Cloud Type,t-14_Clearsky DNI,t-14_Clearsky DHI,t-14_Relative Humidity,t-14_Cloud Type
Timestamp,,,,,,,,,,,,,,,,,,,,,
1-1 0:0,0,169.00,0,0,100.0,0,0,0,95.62,0,...,54.18,0,0,0,54.72,1,0,0,67.66,1
1-1 1:0,0,162.39,0,0,100.0,0,0,0,94.57,1,...,56.23,0,0,0,54.95,1,0,0,66.37,1
1-1 2:0,0,151.00,0,0,100.0,0,0,0,99.62,1,...,59.31,0,0,0,51.60,1,0,0,63.84,1
1-1 3:0,0,138.71,0,0,100.0,0,0,0,96.82,1,...,62.88,0,0,0,48.95,1,0,0,57.88,1
1-1 4:0,0,126.29,0,0,100.0,0,0,0,92.65,1,...,62.31,0,0,0,47.04,1,0,0,56.08,1
